In [ ]:
%load_ext autoreload
%autoreload 2

# Basics: design and rendering

## What we will learn about
- Keeping the design tidy: groups and name conventions
- Design variables
- MiniStudies
- Structure of the optimization notebook

## What we will do
- Transform the resonator length into a design variable
- Write a MiniStudy that isolates the resonator

In [ ]:
# Library imports
import json
from qdesignoptimizer.utils.chip_generation import create_chip_base, ChipType

# Project specific
from design import *
import names as n

## Rendering the design

### 1. Create the chip

In [ ]:
def create_chip_and_gui():
    # Initialize the chip and open the Qiskit Metal GUI.
    chip_type = ChipType(
        size_x="10mm", size_y="10mm", size_z="-300um", material="silicon"
    )
    design, gui = create_chip_base(
        chip_name=n.CHIP_NAME, chip_type=chip_type, open_gui=True
    )

    # Pick the values for the design variables defined in the .json file
    # and add them to the design object created by Qiskit Metal
    with open("design_variables.json") as in_file:
        initial_design_variables = json.load(in_file)
    n.add_design_variables_to_design(design, initial_design_variables)

    return design, gui


design, gui = create_chip_and_gui()

### 2. Call the functions defined in design.py

In [ ]:
def render_qiskit_metal_design(design, gui):
    add_transmon_plus_resonator(design, group=n.NBR_1)
    add_transmon_plus_resonator(design, group=n.NBR_2)

    add_coupler(design)

    add_route_interconnects(design)

    add_launch_pads(design)

    add_chargeline(design, group=n.NBR_1)
    add_chargeline(design, group=n.NBR_2)

    gui.rebuild()
    gui.autoscale()


render_qiskit_metal_design(design, gui)

## Creating the study and rendering the isolated components in Ansys

In [ ]:
# Library imports
from qdesignoptimizer.design_analysis import DesignAnalysis, DesignAnalysisState
from qdesignoptimizer.utils.utils import get_save_path

# Project specific
import mini_studies as ms

In [ ]:
MINI_STUDY = ms.get_mini_study_resonator_only(group=n.NBR_1)
RENDER_QISKIT_METAL = lambda design: render_qiskit_metal_design(design, gui)

design_analysis_state = DesignAnalysisState(design, RENDER_QISKIT_METAL, {})

design_analysis = DesignAnalysis(
    design_analysis_state,
    mini_study=MINI_STUDY,
    opt_targets={},
    save_path=get_save_path("./", n.CHIP_NAME),
    update_design_variables=False,
)
design_analysis.optimize_target({}, {})

In [ ]:
from qdesignoptimizer.utils.utils import close_ansys

close_ansys()